In [ ]:
using Catalyst, DifferentialEquations, Plots, Latexify, StructuralIdentifiability, ModelingToolkit, Sundials

In [ ]:

# rn = @reaction_network begin
#   (k1_f, k1_r), OF <--> IF
#   (k2_f, k2_r), OF + H_out <--> OF_Hb
#   (k3_f, k3_r), OF_Sb <--> OF + S_out
#   (k4_f, k4_r), OF_Hb <--> IF_Hb
#   (k5_f, k5_r), OF_Hb_Sb <--> OF_Hb + S_out
#   (k6_f, k6_r), IF_Sb <--> OF_Sb
#   (k7_f, k7_r), OF_Sb + H_out <--> OF_Hb_Sb
#   (k8_f, k8_r), OF_Hb_Sb <--> IF_Hb_Sb
#   (k9_f, k9_r), IF_Hb <--> IF + H_in
#   (k10_f, k10_r), IF + S_in <--> IF_Sb
#   (k11_f, k11_r), IF_Hb + S_in <--> IF_Hb_Sb
#   (k12_f, k12_r), IF_Hb_Sb <--> IF_Sb + H_in    
# end k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r k7_f k7_r k8_f k8_r k9_f k9_r k10_f k10_r k11_f k11_r k12_f k12_r
# odesys = convert(ODESystem, rn)

function calc_flux(conc_terms, rate_terms, vol)
    IF_Hb_Sb = conc_terms[1]
    IF_Sb = conc_terms[2]
    H_in = conc_terms[3]
    rxn12_k1 = rate_terms[1]
    rxn12_k2 = rate_terms[2]
    x1 = rxn12_k1*IF_Hb_Sb
    x2 = rxn12_k2*IF_Sb.*H_in
    y_pred = vol*(x1-x2)  # returns y_pred            
end


rn2 = @reaction_network begin
  (k1_f, k1_r), OF <--> IF
  (k2_f*H_out, k2_r), OF <--> OF_Hb
  (k3_f, k3_r*S_out), OF_Sb <--> OF
  (k4_f, k4_r), OF_Hb <--> IF_Hb
  (k5_f, k5_r*S_out), OF_Hb_Sb <--> OF_Hb 
  (k6_f, k6_r), IF_Sb <--> OF_Sb
  (k7_f*H_out, k7_r), OF_Sb <--> OF_Hb_Sb
  (k8_f, k8_r), OF_Hb_Sb <--> IF_Hb_Sb
  (k9_f, k9_r), IF_Hb <--> IF + H_in
  (k10_f, k10_r), IF + S_in <--> IF_Sb
  (k11_f, k11_r), IF_Hb + S_in <--> IF_Hb_Sb
  (k12_f, k12_r), IF_Hb_Sb <--> IF_Sb + H_in    
end H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r k7_f k7_r k8_f k8_r k9_f k9_r k10_f k10_r k11_f k11_r k12_f k12_r


odesys2 = convert(ODESystem, rn2)
equations(odesys2)

# k_conf = 1e2
# k_H_on = 1e10
# k_H_off = 1e3
# k_S_on = 1e7
# k_S_off = 1e3

# @parameters  H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r k7_f k7_r k8_f k8_r k9_f k9_r k10_f k10_r k11_f k11_r k12_f k12_r
# @variables t H_in(t) S_in(t) OF(t) OF_Hb(t) OF_Sb(t) OF_Hb_Sb(t) IF(t) IF_Hb(t) IF_Sb(t) IF_Hb_Sb(t)
# pmap  = (H_out => 1e-7, S_out => 1e-3,
#         k1_f =>0, k1_r =>0,
#         k2_f =>k_H_on, k2_r =>k_H_off,
#         k3_f =>k_S_off, k3_r =>k_S_on,
#         k4_f =>k_conf, k4_r =>k_conf,
#         k5_f =>0, k5_r =>0,
#         k6_f =>k_conf, k6_r =>k_conf,
#         k7_f =>0, k7_r =>0,
#         k8_f =>0, k8_r =>0,
#         k9_f =>0, k9_r =>0,
#         k10_f =>0, k10_r =>0,
#         k11_f =>k_S_on, k11_r =>k_S_off,
#         k12_f =>k_H_off, k12_r =>k_H_on,
# )

# u0map = [H_in => 1e-7, S_in => 1e-3, 
#         OF => 2.833e-8, IF => 2.125e-08, 
#         OF_Hb => 2.833e-8, IF_Hb => 2.833e-8, 
#         OF_Sb => 2.125e-08, IF_Sb => 2.125e-08, 
#         OF_Hb_Sb => 2.125e-08, IF_Hb_Sb => 2.833e-8, 

# ]

# tspan = (0.0,10.0)
# oprob = ODEProblem(odesys2, u0map, tspan, pmap)
# event_times = 5.0
# condition(u,t,integrator) = t ∈ event_times
# affect!(integrator) = integrator.p["H_in"] = 5e-8
# cb = DiscreteCallback(condition,affect!)
# #sol = solve(oprob, AutoTsit5(Rosenbrock23()), reltol=1e-16, abstol=1e-18, saveat=0.04,callback=cb, tstops=[5.0])
# sol = solve(oprob, Rosenbrock23(), saveat=0.04,callback=cb, tstops=[5.0])


# # event_times = [5.0,]
# # condition(u,t,integrator) = t ∈ event_times
# # affect!(integrator) = integrator.p[12] = 5e-8
# # cb = DiscreteCallback(condition,affect!)
# # sol = solve(oprob, CVODE_BDF(), saveat=0.04, abstol = 1e-16, reltol = 1e-18, callback=cb, tstops=[5.0])


# # tspan = (0., 5)
# # oprob = ODEProblem(odesys2, u₀map, tspan, pmap)
# # sol = solve(oprob, CVODE_BDF(), reltol=1e-16, abstol=1e-18)

# # calcualte flux 
# # conc_terms = [sol[IF_Hb_Sb],sol[IF_Sb],sol[H_in]]
# # rate_terms = [k_H_off,k_H_on]
# # vol = 0.0001
# # y_pred = calc_flux(conc_terms, rate_terms, vol)
# # plot(y_pred)

In [ ]:
using Catalyst, DifferentialEquations, StructuralIdentifiability, ModelingToolkit

In [ ]:
rn2 = @reaction_network begin
  (k1_f, k1_r), OF <--> IF
  (k2_f*H_out, k2_r), OF <--> OF_Hb
  (k3_f, k3_r*S_out), OF_Sb <--> OF
  (k4_f, k4_r), OF_Hb <--> IF_Hb
  (k5_f, k5_r*S_out), OF_Hb_Sb <--> OF_Hb 
  (k6_f, k6_r), IF_Sb <--> OF_Sb
  (k7_f*H_out, k7_r), OF_Sb <--> OF_Hb_Sb
  (k8_f, k8_r), OF_Hb_Sb <--> IF_Hb_Sb
  (k9_f, k9_r), IF_Hb <--> IF + H_in
  (k10_f, k10_r), IF + S_in <--> IF_Sb
  (k11_f, k11_r), IF_Hb + S_in <--> IF_Hb_Sb
  (k12_f, k12_r), IF_Hb_Sb <--> IF_Sb + H_in    
end H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r k7_f k7_r k8_f k8_r k9_f k9_r k10_f k10_r k11_f k11_r k12_f k12_r


odesys2 = convert(ODESystem, rn2)
equations(odesys2)

In [ ]:





test_ode = @ODEmodel(
    OF'(t) = k1_r*IF(t)+k2_r*OF_Hb(t)+k3_f*OF_Sb(t)−k1_f*OF(t)−H_out*k2_f*OF(t)−S_out*k3_r*OF(t),
    IF'(t) = k9_f*IF_Hb(t)+k10_r*IF_Sb(t)+k1_f*OF(t)−k1_r*IF(t)−k9_r*H_in(t)*IF(t)−k10_f*IF(t)*S_in(t),
    OF_Hb'(t) = k4_r*IF_Hb(t)+k5_f*OF_Hb_Sb(t)+H_out*k2_f*OF(t)−k2_r*OF_Hb(t)−k4_f*OF_Hb(t)−S_out*k5_r*OF_Hb(t),
    OF_Sb'(t) = k6_f*IF_Sb(t)+k7_r*OF_Hb_Sb(t)+S_out*k3_r*OF(t)−k3_f*OF_Sb(t)−k6_r*OF_Sb(t)−H_out*k7_f*OF_Sb(t),
    IF_Hb'(t) = k11_r*IF_Hb_Sb(t)+k4_f*OF_Hb(t)−k4_r*IF_Hb(t)−k9_f*IF_Hb(t)+k9_r*H_in(t)*IF(t)−k11_f*IF_Hb(t)*S_in(t),
    OF_Hb_Sb'(t) = k8_r*IF_Hb_Sb(t)+S_out*k5_r*OF_Hb(t)−k5_f*OF_Hb_Sb(t)+H_out*k7_f*OF_Sb(t)−k7_r*OF_Hb_Sb(t)−k8_f*OF_Hb_Sb(t),
    IF_Sb'(t) = k12_f*IF_Hb_Sb(t)+k6_r*OF_Sb(t)−k10_r*IF_Sb(t)−k6_f*IF_Sb(t)+k10_f*IF(t)*S_in(t)−k12_r*H_in(t)*IF_Sb(t),
    IF_Hb_Sb'(t) = k8_f*OF_Hb_Sb(t)−k11_r*IF_Hb_Sb(t)−k12_f*IF_Hb_Sb(t)−k8_r*IF_Hb_Sb(t)+k12_r*H_in(t)*IF_Sb(t)+k11_f*IF_Hb(t)*S_in(t),
    H_in'(t) = k9_f*IF_Hb(t)+k12_f*IF_Hb_Sb(t)−k12_r*H_in(t)*IF_Sb(t)−k9_r*H_in(t)*IF(t),
    S_in'(t) = k10_r*IF_Sb(t)+k11_r*IF_Hb_Sb(t)−k10_f*IF(t)*S_in(t)−k11_f*IF_Hb(t)*S_in(t),
    y1(t) = H_out, 
    y2(t) = S_out, 
    y3(t) = H_in'(t)
)

@time global_id = assess_local_identifiability(test_ode,0.999, :SE)
println(global_id)



### Transporter reaction network

In [ ]:
using Catalyst, DifferentialEquations, StructuralIdentifiability, ModelingToolkit, Noise, Plots, DataFrames, CSV

# define reaction network
rn = @reaction_network begin
  (k1_f, k1_r), OF <--> IF
  (k2_f*H_out, k2_r), OF <--> OF_Hb
  (k3_f, k3_r*S_out), OF_Sb <--> OF
  (k4_f, k4_r), OF_Hb <--> IF_Hb
  (k5_f, k5_r*S_out), OF_Hb_Sb <--> OF_Hb 
  (k6_f, k6_r), IF_Sb <--> OF_Sb
  (k7_f*H_out, k7_r), OF_Sb <--> OF_Hb_Sb
  (k8_f, k8_r), OF_Hb_Sb <--> IF_Hb_Sb
  (k9_f, k9_r), IF_Hb <--> IF + H_in
  (k10_f, k10_r), IF + S_in <--> IF_Sb
  (k11_f, k11_r), IF_Hb + S_in <--> IF_Hb_Sb
  (k12_f, k12_r), IF_Hb_Sb <--> IF_Sb + H_in    
end H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r k7_f k7_r k8_f k8_r k9_f k9_r k10_f k10_r k11_f k11_r k12_f k12_r

# convert reaction network to ODEsystem
odesys = convert(ODESystem, rn)

# set reference values for parameters (rate constants and external H and S concentrations)
k_conf = 1e2
k_H_on = 1e10
k_H_off = 1e3
k_S_on = 1e7
k_S_off = 1e3
H_out_ref = 1e-7
S_out_ref = 1e-3

# set parameter/variable symbols and mapping
@parameters  H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r k7_f k7_r k8_f k8_r k9_f k9_r k10_f k10_r k11_f k11_r k12_f k12_r
@variables t H_in(t) S_in(t) OF(t) OF_Hb(t) OF_Sb(t) OF_Hb_Sb(t) IF(t) IF_Hb(t) IF_Sb(t) IF_Hb_Sb(t)
pmap  = [H_out => H_out_ref, S_out => S_out_ref,
        k1_f =>0, k1_r =>0,
        k2_f =>k_H_on, k2_r =>k_H_off,
        k3_f =>k_S_off, k3_r =>k_S_on,
        k4_f =>k_conf, k4_r =>k_conf,
        k5_f =>0, k5_r =>0,
        k6_f =>k_conf, k6_r =>k_conf,
        k7_f =>0, k7_r =>0,
        k8_f =>0, k8_r =>0,
        k9_f =>0, k9_r =>0,
        k10_f =>0, k10_r =>0,
        k11_f =>k_S_on, k11_r =>k_S_off,
        k12_f =>k_H_off, k12_r =>k_H_on,
]

u0map = [H_in => 1e-7, S_in => 1e-3, 
        OF => 2.833e-8, IF => 2.125e-08, 
        OF_Hb => 2.833e-8, IF_Hb => 2.833e-8, 
        OF_Sb => 2.125e-08, IF_Sb => 2.125e-08, 
        OF_Hb_Sb => 2.125e-08, IF_Hb_Sb => 2.833e-8, 

]


# create ODEproblem 
tspan = (0.0,15.0)
oprob = ODEProblem(odesys, u0map, tspan, pmap)

#testing
probSS = SteadyStateProblem(rn,u0map, p=pmap )
sol = solve(prob,SSRootfind())

# # set timed events
# event_times = [5.0,10]
# condition(u,t,integrator) = t ∈ event_times
# function affect!(integrator)
#     if integrator.t == 5.0
#         integrator.p[1] = 5e-8
#     elseif integrator.t == 10.0
#         integrator.p[1] = 1e-7
#     end
# end
# #affect!(integrator) = integrator.p[:H_out] = 5e-8 
# cb = DiscreteCallback(condition,affect!)

# # solve ODE system
# sol = solve(oprob, TRBDF2(), abstol = 1e-14, reltol = 1e-6, saveat=0.04, callback=cb, tstops=[5.0, 10.0])

# # calculate flux
# function calc_ion_flux(p,u,v)
    
#     IF_Hb_Sb_tmp 
#     IF_Sb_tmp 
#     H_in_tmp 
#     rxn12_k1_tmp1
#     rxn12_k2_tmp1 

#     x11 = rxn12_k1_tmp1*IF_Hb_Sb_tmp1
#     x21 = rxn12_k2_tmp1*IF_Sb_tmp1.*H_in_tmp1

#     y_pred = vol_tmp1*(x11-x21)
    
#     k9_f = p[1]
#     k9_r = p[2]
    
#     flux_in = v*()
#     net_flux = flux_in - flux_out         
# end
# k = [pmap[19][2], pmap[20][2], pmap[25][2], pmap[26][2]]
# c = [sol[IF_Hb], sol[H_in], sol[IF], sol[IF_Hb_Sb], sol[IF_Sb]]
# vol = 0.0001
# y_true = calc_ion_flux(k,c,vol)

# # add noise to create synthetic flux trace
# stdev_ref = 1e-13
# y_obs = add_gauss(y_true,stdev_ref)
# #plot(y_true, legend = false, ylim=[-1.5e-11,1.5e-11]);scatter!(y_obs, alpha = 0.5)

# df = DataFrame(data = y_obs)
# CSV.write("data.csv", df)
# df2 = CSV.read("data.csv", DataFrame)
# y_obs2 = df2[!, "data"]
# scatter(y_obs, alpha = 0.5, ylim=[-1.5e-11,1.5e-11])

### Sampling

In [ ]:
using Turing

Turing.setadbackend(:forwarddiff)

@model function transporter_sampler(data, oprob)
    # set priors
    stdev_sample ~ Uniform(0.5e-13 , 1.25e-13) # ~ is the tilde character
    log_k2_f ~ Uniform(6, 12)  # H on rate (1e10)
    log_k2_r ~ Uniform(-1,5)  # H off rate (1e3)
    log_k3_f ~ Uniform(-1,5)  # S off rate (1e3)
    log_k3_r ~ Uniform(3,9)  # S on rate (1e7)
    log_k4_f ~ Uniform(-2,4)  # conf rate (1e2)
    log_k4_r ~ Uniform(-2,4)   # conf rate (1e2)
    log_k6_f ~ Uniform(-2,4)   # conf rate (1e2)
    log_k6_r ~ Uniform(-2,4)   # conf rate (1e2)
    log_k11_f ~ Uniform(3,9) # S on rate (1e7)
    log_k11_r ~ Uniform(-1,5)  # S off rate (1e3)
    log_k12_f ~ Uniform(-1,5)  # H off rate (1e3)
    log_k12_r = (log_k2_f+log_k3_f+log_k4_f+log_k6_f+log_k11_f+log_k12_f)-(log_k2_r+log_k3_r+log_k4_r+log_k6_r+log_k11_r)
     
    # set parameter values
    pmap1 = [H_out => H_out_ref, S_out => S_out_ref,
            k1_f =>0.0, k1_r =>0.0,
            k2_f =>10.0^log_k2_f, k2_r =>10.0^log_k2_r,
            k3_f =>10.0^log_k3_f, k3_r =>10.0^log_k3_r,
            k4_f =>10.0^log_k4_f, k4_r =>10.0^log_k4_r,
            k5_f =>0.0, k5_r =>0.0,
            k6_f =>10.0^log_k6_f, k6_r =>10.0^log_k6_r,
            k7_f =>0.0, k7_r =>0.0,
            k8_f =>0.0, k8_r =>0.0,
            k9_f =>0.0, k9_r =>0.0,
            k10_f =>0.0, k10_r =>0.0,
            k11_f =>10.0^log_k11_f, k11_r =>10.0^log_k11_r,
            k12_f =>10.0^log_k12_f, k12_r =>10.0^log_k12_r,
    ]

    # create ODEproblem (same u0map and tspan as synthetic model)
    oprob1 = ODEProblem(odesys, u0map, tspan, pmap1)

    # set timed events
    event_times = [5.0,10]
    condition(u,t,integrator) = t ∈ event_times
    function affect!(integrator)
        if integrator.t == 5.0
            integrator.p[1] = 5e-8
        elseif integrator.t == 10.0
            integrator.p[1] = 1e-7
        end
    end
    cb = DiscreteCallback(condition,affect!)

    # solve ODE system
    sol1 = solve(oprob1, TRBDF2(), abstol = 1e-14, reltol = 1e-6, saveat=0.04, progress=false, callback=cb, tstops=[5.0, 10.0])

    # calc flux
    k1 = [pmap1[19][2], pmap1[20][2], pmap1[25][2], pmap1[26][2]]
    c1 = [sol1[IF_Hb], sol1[H_in], sol1[IF], sol1[IF_Hb_Sb], sol1[IF_Sb]]
    vol1 = 0.0001
    y_pred = calc_ion_flux(k1,c1,vol1)
    
    for i = 1:length(y_pred)
        data[i] ~ Normal(y_pred[i], stdev_sample)
    end
    
end

model = transporter_sampler(y_obs, oprob)

c = sample(model, NUTS(10000, .65),10000);

In [ ]:
c[:stdev_sample]

In [ ]:
c

In [ ]:
using StatsPlots
plot(c)

# 1:1 transporter - single cycle model

#### load and plot data

In [ ]:
using Plots, DataFrames, CSV 

# load and plot data
df = CSV.read("data.csv", DataFrame)
y_obs = df[!, "data"]
plot(y_obs, legend = false, ylim=[-1.5e-11,1.5e-11]) ;scatter!(y_obs, legend = false, alpha = 0.5, ylim=[-1.5e-11,1.5e-11])

#### build catalyst model

In [ ]:
using Catalyst, DifferentialEquations, StatsPlots, Plots, DataFrames, CSV, Random, Turing, Latexify, ModelingToolkit, DynamicHMC
using ModelingToolkit: varmap_to_vars
using Optim
Random.seed!(14)

# load flux dataset of ~ 300 data points
df = CSV.read("data_1c.csv", DataFrame) 
y_obs = df[!, "data"]

# Normal log-likelihood calculation
function calc_norm_log_likelihood(y_obs,y_pred,sigma)
    n = length(y_obs)
    t1 = -(n/2) * (log(2*pi))
    t2 = -(n/2) * (sigma^2)
    t3_sum = 0
    for i in 1:n
        t3_sum = t3_sum + (y_obs[i]-y_pred[i])^2
    end
    t3 = -(1/(2*sigma^2)) * t3_sum
    log_likelihood = t1+t2+t3
end


# flux calculation function
function calc_flux(p,u,v)
    flux_in = v*(p[1]*u[1])
    flux_out = v*(p[2]*u[2].*u[3])
    net_flux = flux_in - flux_out         
end

# define reaction network in Catalyst
rn = @reaction_network begin
  (k1_f*H_out, k1_r), OF <--> OF_Hb
  (k2_f, k2_r), OF_Hb <--> IF_Hb
  (k3_f, k3_r), IF_Hb + S_in <--> IF_Hb_Sb
  (k4_f, k4_r), IF_Hb_Sb <--> IF_Sb + H_in
  (k5_f, k5_r), IF_Sb <--> OF_Sb
  (k6_f, k6_r*S_out), OF_Sb <--> OF
end H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r
@parameters  H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r stdev_s
@variables t H_in(t) S_in(t) OF(t) OF_Hb(t) IF_Hb(t) IF_Hb_Sb(t) IF_Sb(t) OF_Sb(t)
u0map = [H_in => 1e-7, S_in => 1e-3, 
        OF => 2.833e-8, OF_Hb => 2.833e-8,  
        IF_Hb => 2.833e-8, IF_Hb_Sb => 2.833e-8,
        IF_Sb => 2.125e-8, OF_Sb => 2.125e-8, 
        ]
H_out_ref = 1e-7
S_out_ref = 1e-3
pmap = [H_out => H_out_ref, S_out => S_out_ref,
        k1_f =>1e10, k1_r =>1e3,
        k2_f =>1e2, k2_r =>1e2,
        k3_f =>1e7, k3_r =>1e3,
        k4_f =>1e3, k4_r =>1e10,
        k5_f =>1e2, k5_r =>1e2,
        k6_f =>1e3, k6_r =>1e7
        ]

# convert reaction network to ODEsystem
odesys = convert(ODESystem, rn)
latexify(odesys)


# create ODEproblem w/ timed events discrete callback
tspan1 = (0.0,15.0)
oprob = ODEProblem(odesys, u0map, tspan1, pmap)
event_times = [5.0,10]
condition(u,t,integrator) = t ∈ event_times
function affect!(integrator)
    if integrator.t == 5.0
        integrator.p[1] = 5e-8
    elseif integrator.t == 10.0
        integrator.p[1] = 1e-7
    end
end
cb = DiscreteCallback(condition,affect!)

### testing
# calc flux (y_pred)
sol1 = solve(oprob, TRBDF2(), abstol = 1e-19, reltol = 1e-17, saveat=0.04, callback=cb, tstops=[5.0, 10.0])
k1 = [pmap[9][2], pmap[10][2]]
c1 = [sol1[IF_Hb_Sb], sol1[H_in], sol1[IF_Sb]]
vol1 = 0.0001
y_pred = calc_flux(k1,c1,vol1)
df = DataFrame(data = y_pred)
CSV.write("data_ideal.csv", df)
### 



# create Turing model w/ forward differentiation
Turing.setadbackend(:forwarddiff)
@model function rxn_sampler(data, prob, sys)
    
    # set priors
    stdev_sample ~ Uniform(0.5e-13 , 1.25e-13) 
    log_k1_f ~ Uniform(6, 12)  # log10 rate constant (ref=1e10)
    log_k1_r ~ Uniform(-1,5)  # log10 rate constant (ref=1e3)  
    log_k2_f ~ Uniform(-2,4)  # log10 rate constant (ref=1e2)
    log_k2_r ~ Uniform(-2,4)  # log10 rate constant (ref=1e2)
    log_k3_f ~ Uniform(3,9)  # log10 rate constant (ref=1e7) 
    log_k3_r ~ Uniform(-1,5)  # log10 rate constant (ref=1e3)  
    log_k4_f ~ Uniform(-1,5)  # log10 rate constant (ref=1e3) 
    log_k4_r ~ Uniform(6, 12)  # log10 rate constant (ref=1e10)
    log_k5_f ~ Uniform(-2,4)  # log10 rate constant (ref=1e2)
    log_k5_r ~ Uniform(-2,4)   # log10 rate constant (ref=1e2)
    log_k6_f ~ Uniform(-1,5)  # log10  rate constant (ref=1e3)
    
    # set parameter values and initial concentrations
    log_k6_r = (log_k1_f+log_k2_f+log_k3_f+log_k4_f+log_k5_f+log_k6_f)-(log_k1_r+log_k2_r+log_k3_r+log_k4_r+log_k5_r) 
    pmap1 = [H_out => H_out_ref, S_out => S_out_ref,
            k1_f =>10.0^log_k1_f, k1_r =>10.0^log_k1_r,
            k2_f =>10.0^log_k2_f, k2_r =>10.0^log_k2_r,
            k3_f =>10.0^log_k3_f, k3_r =>10.0^log_k3_r,
            k4_f =>10.0^log_k4_f, k4_r =>10.0^log_k4_r,
            k5_f =>10.0^log_k5_f, k5_r =>10.0^log_k5_f,
            k6_f =>10.0^log_k6_f, k6_r =>10.0^log_k6_r,
    ]
    pnew = varmap_to_vars(pmap1, parameters(sys))
    
    oprob1 = remake(prob, p=pnew)
    sol = solve(oprob1, TRBDF2(), abstol = 1e-19, reltol = 1e-17, saveat=0.04, callback=cb, tstops=[5.0, 10.0])
    
    # calc flux (y_pred)
    k = [pmap1[9][2], pmap1[10][2]]
    c = [sol[IF_Hb_Sb], sol[H_in], sol[IF_Sb]]
    vol = 0.0001
    y_pred = calc_flux(k,c,vol)
      
    for i = 1:length(data)
        data[i] ~ Normal(y_pred[i], stdev_sample)
    end
end
model = rxn_sampler(y_obs, oprob, odesys)

p2 = [1e-13,
      10, 3,
      2, 2,
      7, 3,
      3, 10,
      2, 2,
      3, 
        ]

iterations = 1000
n_adapt_step = 100
ϵ = 1.0
τ = 1000

chain = sample(model, NUTS(0.65), iterations)
#chain = sample(model, NUTS(100, 0.65), iterations, init_params = p2)
#chain = sample(model, NUTS(1000, 0.65), iterations)
#chain = sample(model, HMC(ϵ, τ), iterations)
# run single-chain No-U-Turn sampler 
#c = sample(model, NUTS(.65), iterations, init_params=pmap_test)
#c = sample(model, DynamicNUTS(), MCMCThreads(), 10000, 4, progress=true)
#chain = mapreduce(c -> sample(model, DynamicNUTS() ,10000), chainscat, 1:3)

#chain = sample(model, MH(0.65), iterations, init_params = p2)

In [ ]:
println(chain["stdev_sample"])

In [ ]:
p = plot(chain)

In [ ]:
savefig(p,"file7.png")

In [ ]:
c["stdev_sample"]

### Testing NUTS sampler


In [ ]:
using Catalyst, DifferentialEquations, StatsPlots, Plots, DataFrames, CSV, Random, Turing, Latexify, ModelingToolkit, DynamicHMC, Sundials
using ModelingToolkit: varmap_to_vars
using Optim
Random.seed!(14)

# load flux dataset fo ~ 300 data points
df = CSV.read("data_ideal.csv", DataFrame) 
y_obs = df[!, "data"]

# Normal log-likelihood calculation
function calc_norm_log_likelihood(y_obs,y_pred,sigma)
    n = length(y_obs)
    t1 = -(n/2) * (log(2*pi))
    t2 = -(n/2) * (sigma^2)
    t3_sum = 0
    for i in 1:n
        t3_sum = t3_sum + (y_obs[i]-y_pred[i])^2
    end
    t3 = -(1/(2*sigma^2)) * t3_sum
    log_likelihood = t1+t2+t3
end

# flux calculation function
function calc_flux(p,u,v)
    flux_in = v*(p[1]*u[1])
    flux_out = v*(p[2]*u[2].*u[3])
    net_flux = flux_in - flux_out         
end

# define reaction network in Catalyst
rn = @reaction_network begin
  (k1_f*H_out, k1_r), OF <--> OF_Hb
  (k2_f, k2_r), OF_Hb <--> IF_Hb
  (k3_f, k3_r), IF_Hb + S_in <--> IF_Hb_Sb
  (k4_f, k4_r), IF_Hb_Sb <--> IF_Sb + H_in
  (k5_f, k5_r), IF_Sb <--> OF_Sb
  (k6_f, k6_r*S_out), OF_Sb <--> OF
end H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r
@parameters  H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r stdev_s
@variables t H_in(t) S_in(t) OF(t) OF_Hb(t) IF_Hb(t) IF_Hb_Sb(t) IF_Sb(t) OF_Sb(t)
u0map = [H_in => 1e-7, S_in => 1e-3, 
        OF => 2.833e-8, OF_Hb => 2.833e-8,  
        IF_Hb => 2.833e-8, IF_Hb_Sb => 2.833e-8,
        IF_Sb => 2.125e-8, OF_Sb => 2.125e-8, 
        ]
H_out_ref = 1e-7
S_out_ref = 1e-3

#pmap = [H_out => H_out_ref, S_out => S_out_ref,
#       k1_f =>10^8.2048, k1_r =>10^3.8047,
#       k2_f =>10^2.2232, k2_r =>10^-0.0276,
#       k3_f =>10^6.2126, k3_r =>10^4.0139,
#       k4_f =>10^0.3190, k4_r =>10^6.9358,
#       k5_f =>10^2.9380, k5_r =>10^-1.0432,
#       k6_f =>10^1.6900, k6_r =>10^k_tmp
#       ]

Random.seed!(9)

#println(6*rand()+6)


k1_f_tmp = 10 #10^(6*rand()+6) #10^8.2048 #1e10 #
k1_r_tmp = 3 #10^(6*rand()-1) #10^3.8047 #1e3 #
k2_f_tmp = 2 #10^(6*rand()-2) #10^2.2232 #1e2 #
k2_r_tmp = 2 #10^(6*rand()-2) #10^-0.0276 #1e2 #
k3_f_tmp = 7 #10^(6*rand()+3) #10^6.2126 #1e7 #
k3_r_tmp = 3 #10^(6*rand()-1) #10^4.0139 #1e3 #
k4_f_tmp = 3 #10^(6*rand()-1) #10^0.3190 #1e3 #
k4_r_tmp = 10 #10^(6*rand()+6) #10^6.9358 #1e10 # 
k5_f_tmp = 2 #10^(6*rand()-2) #10^2.9380 #1e2 #
k5_r_tmp = 2 #10^(6*rand()-2) #10^-1.0432 #1e2 #
k6_f_tmp = 3 #10^(6*rand()-1) #10^1.6900 #1e3 #
k6_r_tmp = (k1_f_tmp+k2_f_tmp+k3_f_tmp+k4_f_tmp+k5_f_tmp+k6_f_tmp)-(k1_r_tmp+k2_r_tmp+k3_r_tmp+k4_r_tmp+k5_r_tmp)
println(k6_r_tmp)
println(k1_f_tmp*k2_f_tmp*k3_f_tmp*k4_f_tmp*k5_f_tmp*k6_f_tmp)
println(k1_r_tmp*k2_r_tmp*k3_r_tmp*k4_r_tmp*k5_r_tmp)

pmap = [H_out => H_out_ref, S_out => S_out_ref,       
        k1_f =>10^k1_f_tmp, k1_r =>10^k1_r_tmp,
        k2_f =>10^k2_f_tmp, k2_r =>10^k2_r_tmp,        
        k3_f =>10^k3_f_tmp, k3_r =>10^k3_r_tmp,        
        k4_f =>10^k4_f_tmp, k4_r =>10^k4_r_tmp,
        k5_f =>10^k5_f_tmp, k5_r =>10^k5_r_tmp,
        k6_f =>10^k6_f_tmp, k6_r =>10^k6_r_tmp
       ]
       
df = DataFrame(
        "k1_f" =>10^k1_f_tmp, "k1_r" =>10^k1_r_tmp,
        "k2_f" =>10^k2_f_tmp, "k2_r" =>10^k2_r_tmp,        
        "k3_f" =>10^k3_f_tmp, "k3_r" =>10^k3_r_tmp,        
        "k4_f" =>10^k4_f_tmp, "k4_r" =>10^k4_r_tmp,
        "k5_f" =>10^k5_f_tmp, "k5_r" =>10^k5_r_tmp,
        "k6_f" =>10^k6_f_tmp, "k6_r" =>10^k6_r_tmp
)

println(df)
CSV.write("pmap_rand.csv", df)

# convert reaction network to ODEsystem
odesys = convert(ODESystem, rn)
latexify(odesys)


# create ODEproblem w/ timed events discrete callback
tspan1 = (0.0,15.0)
oprob = ODEProblem(odesys, u0map, tspan1, pmap)
event_times = [5.0,10]
condition(u,t,integrator) = t ∈ event_times
function affect!(integrator)
    if integrator.t == 5.0
        integrator.p[1] = 5e-8
    elseif integrator.t == 10.0
        integrator.p[1] = 1e-7
    end
end
cb = DiscreteCallback(condition,affect!)

### testing
# calc flux (y_pred)
sol1 = solve(oprob, TRBDF2(), abstol = 1e-16, reltol = 1e-10, saveat=0.04, callback=cb, tstops=[5.0, 10.0])
k1 = [pmap[9][2], pmap[10][2]]
c1 = [sol1[IF_Hb_Sb], sol1[H_in], sol1[IF_Sb]]
vol1 = 0.0001
y_pred = calc_flux(k1,c1,vol1)
df = DataFrame(sol1)
CSV.write("data_pop_rand.csv", df)
### 

#sol_ss = solve(oprob,SSRootfind())
#println(sol_ss[end])
#println(sol1[1])
plot(sol1, ylim=[-1e-8,1.75e-7])
#plot(y_pred)

sigma_ref = 1e-13


log_likelihood = calc_norm_log_likelihood(y_obs,y_pred,sigma_ref)
print(log_likelihood)

### debugging model

$\epsilon$

In [ ]:
using Catalyst, DifferentialEquations, StatsPlots, Plots, DataFrames, CSV, Random, Turing, Latexify, ModelingToolkit, DynamicHMC
using ModelingToolkit: varmap_to_vars
using Optim, Noise, AdvancedMH
Random.seed!(14)


# load flux dataset of ~ 300 data points
df = CSV.read("data_1c.csv", DataFrame) 
y_obs = df[!, "data"]


# flux calculation function
function calc_flux(p,u,v)
    flux_in = v*(p[1]*u[1])
    flux_out = v*(p[2]*u[2].*u[3])
    net_flux = flux_in - flux_out         
end

# define reaction network in Catalyst
rn = @reaction_network begin
  (k1_f*H_out, k1_r), OF <--> OF_Hb
  (k2_f, k2_r), OF_Hb <--> IF_Hb
  (k3_f, k3_r), IF_Hb + S_in <--> IF_Hb_Sb
  (k4_f, k4_r), IF_Hb_Sb <--> IF_Sb + H_in
  (k5_f, k5_r), IF_Sb <--> OF_Sb
  (k6_f, k6_r*S_out), OF_Sb <--> OF
end H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r

# convert reaction network to ODEsystem
odesys = convert(ODESystem, rn)
latexify(odesys)

# define initial variable values and parameters
@parameters  H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r stdev_s
@variables t H_in(t) S_in(t) OF(t) OF_Hb(t) IF_Hb(t) IF_Hb_Sb(t) IF_Sb(t) OF_Sb(t)
u0map = [H_in => 1e-7, S_in => 1e-3, 
        OF => 2.833e-8, OF_Hb => 2.833e-8,  
        IF_Hb => 2.833e-8, IF_Hb_Sb => 2.833e-8,
        IF_Sb => 2.125e-8, OF_Sb => 2.125e-8, 
        ]
H_out_ref = 1e-7
S_out_ref = 1e-3
pmap = [H_out => H_out_ref, S_out => S_out_ref,
        k1_f =>1e10, k1_r =>1e3,
        k2_f =>1e2, k2_r =>1e2,
        k3_f =>1e7, k3_r =>1e3,
        k4_f =>1e3, k4_r =>1e10,
        k5_f =>1e2, k5_r =>1e2,
        k6_f =>1e3, k6_r =>1e7
        ]
stdev_ref = 1e-13

# create ODEproblem w/ timed events discrete callback
#tspan1 = (0.0,15.0)
tspan1 = (0.0,7.5)
oprob = ODEProblem(odesys, u0map, tspan1, pmap)
#event_times = [5.0,10]
event_times = [2.5,5.0]
condition(u,t,integrator) = t ∈ event_times
function affect!(integrator)
    #if integrator.t == 5.0
    if integrator.t == 2.5
        integrator.p[1] = 5e-8
    #elseif integrator.t == 10.0
    elseif integrator.t == 5.0
        integrator.p[1] = 1e-7
    end
end
cb = DiscreteCallback(condition,affect!)

### testing
s_test = solve(oprob, TRBDF2(), abstol = 1e-18, reltol = 1e-12, saveat=0.04, callback=cb, tstops=event_times)
k = [pmap[9][2], pmap[10][2]]
c = [s_test[IF_Hb_Sb], s_test[H_in], s_test[IF_Sb]]
vol = 0.0001
y_pred_test = calc_flux(k,c,vol)

# add noise to create synthetic flux trace
stdev_test = 1e-13
y_obs_test = add_gauss(y_pred_test,stdev_test)
### testing

# create Turing model w/ forward differentiation
Turing.setadbackend(:forwarddiff)
@model function rxn_sampler(data, prob, sys)
    
    # set priors
    #log_stdev ~ Uniform(5, 12.5)
    #stdev_sample = 1e-13
    #stdev_sample ~ Uniform(0.25e-13 , 2.5e-13) 
    #stdev_sample ~ InverseGamma(1e-13, 1e-13)
    log_k1_f ~ Uniform(6, 12)  # log10 rate constant (ref=1e10)
    log_k1_r ~ Uniform(-1,5)  # log10 rate constant (ref=1e3)  
    log_k2_f ~ Uniform(-2,4)  # log10 rate constant (ref=1e2)
    log_k2_r ~ Uniform(-2,4)  # log10 rate constant (ref=1e2)
    log_k3_f ~ Uniform(3,9)  # log10 rate constant (ref=1e7) 
    log_k3_r ~ Uniform(-1,5)  # log10 rate constant (ref=1e3)  
    log_k4_f ~ Uniform(-1,5)  # log10 rate constant (ref=1e3) 
    log_k4_r ~ Uniform(6, 12)  # log10 rate constant (ref=1e10)
    log_k5_f ~ Uniform(-2,4)  # log10 rate constant (ref=1e2)
    log_k5_r ~ Uniform(-2,4)   # log10 rate constant (ref=1e2)
    log_k6_f ~ Uniform(-1,5)  # log10  rate constant (ref=1e3)
    
    # set parameter values and initial concentrations
    log_k6_r = (log_k1_f+log_k2_f+log_k3_f+log_k4_f+log_k5_f+log_k6_f)-(log_k1_r+log_k2_r+log_k3_r+log_k4_r+log_k5_r) 
    pmap1 = [H_out => H_out_ref, S_out => S_out_ref,
            k1_f =>10.0^log_k1_f, k1_r =>10.0^log_k1_r,
            k2_f =>10.0^log_k2_f, k2_r =>10.0^log_k2_r,
            k3_f =>10.0^log_k3_f, k3_r =>10.0^log_k3_r,
            k4_f =>10.0^log_k4_f, k4_r =>10.0^log_k4_r,
            k5_f =>10.0^log_k5_f, k5_r =>10.0^log_k5_f,
            k6_f =>10.0^log_k6_f, k6_r =>10.0^log_k6_r,
    ]
    
    # solve ODE system
    pnew = varmap_to_vars(pmap1, parameters(sys))
    oprob1 = remake(prob, p=pnew)
    sol = solve(oprob1, TRBDF2(), abstol = 1e-18, reltol = 1e-12, saveat=0.04, callback=cb, tstops=event_times)
    
    # calc flux (y_pred)
    k = [pmap1[9][2], pmap1[10][2]]
    c = [sol[IF_Hb_Sb], sol[H_in], sol[IF_Sb]]
    vol = 0.0001
    y_pred = calc_flux(k,c,vol)
    #stdev_sample = log_stdev*1e-14
    #stdev_sample = 1e-13
    stdev_sample = 1e-13
    for i = 1:length(data)
        data[i] ~ Normal(y_pred[i], stdev_sample)
    end
end
model = rxn_sampler(y_obs_test, oprob, odesys)

p2 = [
      10.0, 3.0,
      2.0, 2.0,
      7.0, 3.0,
      3.0, 10.0,
      2.0, 2.0,
      3.0, 
        ]
    
iterations = 100_000

#chain = sample(model, NUTS(0.65, init_ϵ=1.1444091796875e-5), iterations)
#chain = sample(model, HMC(1e-6, 100), iterations)
#chain = sample(model, NUTS(0.45), iterations)
chain = sample(model, 
               IS(), 
               init_params=p2, 
               iterations
               )


In [ ]:
plot(chain[:lp], label="log p")

In [ ]:
plot(chain)
#println(chain[:log_stdev][1])
println(chain[:log_k1_f][1])
println(chain[:log_k1_r][1])
println(chain[:log_k2_f][1])
println(chain[:log_k2_r][1])
println(chain[:log_k3_f][1])
println(chain[:log_k3_r][1])
println(chain[:log_k4_f][1])
println(chain[:log_k4_r][1])
println(chain[:log_k5_f][1])
println(chain[:log_k5_r][1])
println(chain[:log_k6_f][1])
println(chain[:lp][1])



In [ ]:
plot(chain)
#png("file5.png")

In [ ]:
#df = DataFrame(chain)
#CSV.write("chain_MH_init_near_global_min.csv", df)
#print(chain)

In [ ]:
#df = CSV.read("data_1c.csv", DataFrame) 
#y_obs = df[!, "data"]
plot(y_pred_test, legend = true, ylim=[-1.5e-11,1.5e-11]) ;scatter!(y_obs_test, legend = true, alpha = 0.5, ylim=[-1.5e-11,1.5e-11])

### 2D likelihood plot

single cycle, 12 parameter model. 

keeping sigma and all rate constants fixed to synthetic values, except for log_k4_f and log_k4_r (used in flux calculation)

In [6]:
using Catalyst, DifferentialEquations, StatsPlots, Plots, DataFrames, CSV, Random, Turing, Latexify, ModelingToolkit, DynamicHMC
using ModelingToolkit: varmap_to_vars
using Optim, Noise, AdvancedMH
Random.seed!(14)
sigma_f = 1
t_short = false

# Normal log-likelihood calculation
function calc_norm_log_likelihood(mu,sigma,X)
    # Normal log-likelihood function: -[(n/2)ln(2pp*sigma^2)]-[sum((X-mu)^2)/(2*sigma^2)]
    # ref: https://www.statlect.com/fundamentals-of-statistics/normal-distribution-maximum-likelihood 
    n = length(X)
    f1 = -1*(n/2)*log(2*pi*sigma^2)
    f2_a = -1/(2*sigma^2)
    f2_b = 0 
    for i in 1:n
        f2_b += (X[i]-mu[i])^2
    end
    f2 = f2_a*f2_b
    log_likelihood = f1+f2
end

# flux calculation function
function calc_flux(p,u,v)
    flux_in = v*(p[1]*u[1])
    flux_out = v*(p[2]*u[2].*u[3])
    net_flux = flux_in - flux_out         
end

# define reaction network in Catalyst
rn = @reaction_network begin
  (k1_f*H_out, k1_r), OF <--> OF_Hb
  (k2_f, k2_r), OF_Hb <--> IF_Hb
  (k3_f, k3_r), IF_Hb + S_in <--> IF_Hb_Sb
  (k4_f, k4_r), IF_Hb_Sb <--> IF_Sb + H_in
  (k5_f, k5_r), IF_Sb <--> OF_Sb
  (k6_f, k6_r*S_out), OF_Sb <--> OF
end H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r

# convert reaction network to ODEsystem
odesys = convert(ODESystem, rn)

# define initial variable values and parameters
H_out_ref = 1e-7
S_out_ref = 1e-3
stdev_ref = sigma_f*1e-13
@parameters  H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r stdev_s
@variables t H_in(t) S_in(t) OF(t) OF_Hb(t) IF_Hb(t) IF_Hb_Sb(t) IF_Sb(t) OF_Sb(t)
u0map = [H_in => 1e-7, S_in => 1e-3, 
        OF => 2.833e-8, OF_Hb => 2.833e-8,  
        IF_Hb => 2.833e-8, IF_Hb_Sb => 2.833e-8,
        IF_Sb => 2.125e-8, OF_Sb => 2.125e-8, 
        ]
pmap = [H_out => H_out_ref, S_out => S_out_ref,
        k1_f =>1e10, k1_r =>1e3,
        k2_f =>1e2, k2_r =>1e2,
        k3_f =>1e7, k3_r =>1e3,
        k4_f =>1e3, k4_r =>1e10,
        k5_f =>1e2, k5_r =>1e2,
        k6_f =>1e3, k6_r =>1e7
        ]


# create ODEproblem w/ timed events discrete callback
if t_short == true
    t_sample = 0.4
else
    t_sample = 0.04
end    
tspan1 = (0.0,15)
oprob = ODEProblem(odesys, u0map, tspan1, pmap)
event_times = [5.0,10.0]
condition(u,t,integrator) = t ∈ event_times
function affect!(integrator)
    if integrator.t == 5.0
        integrator.p[1] = 5e-8
    elseif integrator.t == 10.0
        integrator.p[1] = 1e-7
    end
end
cb = DiscreteCallback(condition,affect!)



# get y_true and y_obs
s_true = solve(oprob, TRBDF2(), abstol = 1e-18, reltol = 1e-12, saveat=t_sample, callback=cb, tstops=event_times)
k_true = [pmap[9][2], pmap[10][2]]
c_true = [s_true[IF_Hb_Sb], s_true[H_in], s_true[IF_Sb]]
vol = 0.0001
y_true = calc_flux(k_true,c_true,vol)
y_obs = add_gauss(y_true,stdev_ref)
println(length(y_obs))
println(calc_norm_log_likelihood(y_obs,stdev_ref,y_true))


##### 2d grid
print("2D grid...")
    #log_k1_f ~ Uniform(6, 12)  # log10 rate constant (ref=1e10)
    #log_k1_r ~ Uniform(-1,5)  # log10 rate constant (ref=1e3)  
    #log_k2_f ~ Uniform(-2,4)  # log10 rate constant (ref=1e2)
    #log_k2_r ~ Uniform(-2,4)  # log10 rate constant (ref=1e2)
    #log_k3_f ~ Uniform(3,9)  # log10 rate constant (ref=1e7) 
    #log_k3_r ~ Uniform(-1,5)  # log10 rate constant (ref=1e3)  
    #log_k4_f ~ Uniform(-1,5)  # log10 rate constant (ref=1e3) 
    #log_k4_r ~ Uniform(6, 12)  # log10 rate constant (ref=1e10)
    #log_k5_f ~ Uniform(-2,4)  # log10 rate constant (ref=1e2)
    #log_k5_r ~ Uniform(-2,4)   # log10 rate constant (ref=1e2)
    #log_k6_f ~ Uniform(-1,5)  # log10  rate constant (ref=1e3)'

n_grid = 100
x1_range = LinRange(-1, 5,n_grid) # ref = 10
x2_range = LinRange(6, 12,n_grid) # ref = 3
z = zeros(n_grid,n_grid) 
i_idx = 0
max_log_like = -Inf

for i in x1_range
    i_idx += 1
    j_idx = 0
    log_k_f_tmp = i  
    
    for j in x2_range
        j_idx += 1
        log_k_r_tmp = j  

        # set parameter values and mapping
        sigma = sigma_f*1e-13
        log_k1_f = 10  # log10 rate constant (ref=1e10)
        log_k1_r = 3   # log10 rate constant (ref=1e3)  
        log_k2_f = 2   # log10 rate constant (ref=1e2)
        log_k2_r = 2  # log10 rate constant (ref=1e2)
        log_k3_f = 7   # log10 rate constant (ref=1e7) 
        log_k3_r = 3  # log10 rate constant (ref=1e3)  
        log_k4_f = log_k_f_tmp # log10 rate constant (ref=1e3) 
        log_k4_r = log_k_r_tmp # log10 rate constant (ref=1e10)
        log_k5_f = 2   # log10 rate constant (ref=1e2)
        log_k5_r = 2   # log10 rate constant (ref=1e2)
        log_k6_f = 3   # log10  rate constant (ref=1e3)
        log_k6_r = (log_k1_f+log_k2_f+log_k3_f+log_k4_f+log_k5_f+log_k6_f)-(log_k1_r+log_k2_r+log_k3_r+log_k4_r+log_k5_r) 
        pmap1 = [H_out => H_out_ref, S_out => S_out_ref,
                    k1_f =>10.0^log_k1_f, k1_r =>10.0^log_k1_r,
                    k2_f =>10.0^log_k2_f, k2_r =>10.0^log_k2_r,
                    k3_f =>10.0^log_k3_f, k3_r =>10.0^log_k3_r,
                    k4_f =>10.0^log_k4_f, k4_r =>10.0^log_k4_r,
                    k5_f =>10.0^log_k5_f, k5_r =>10.0^log_k5_f,
                    k6_f =>10.0^log_k6_f, k6_r =>10.0^log_k6_r,
                 ]

        # solve ODE system
        pnew = varmap_to_vars(pmap1, parameters(odesys))
        oprob1 = remake(oprob, p=pnew)
        s_tmp = solve(oprob1, TRBDF2(), abstol = 1e-18, reltol = 1e-12, saveat=t_sample, callback=cb, tstops=event_times)

        # calculate flux
        k_tmp = [pmap1[9][2], pmap1[10][2]]
        c_tmp = [s_tmp[IF_Hb_Sb], s_tmp[H_in], s_tmp[IF_Sb]]
        vol = 0.0001
        y_tmp = calc_flux(k_tmp,c_tmp,vol)

        # calculate log-likelihood
        log_like_tmp = calc_norm_log_likelihood(y_obs,sigma,y_tmp)
        z[i_idx,j_idx] = log_like_tmp
        
        # update max log likelihood if needed
        if log_like_tmp > max_log_like
            max_log_like = log_like_tmp
        end
    end
end
println(max_log_like)
##### end of 2D grid



##### sampling
println("sampling...")
# create Turing model w/ forward differentiation
Turing.setadbackend(:forwarddiff)
setchunksize(400)
@model function rxn_sampler(data, prob, sys)
    
    # set priors
    stdev_sample2 = 1e-13
    log_k1_f =10  # log10 rate constant (ref=1e10)
    log_k1_r =3  # log10 rate constant (ref=1e3)  
    log_k2_f =2  # log10 rate constant (ref=1e2)
    log_k2_r =2  # log10 rate constant (ref=1e2)
    log_k3_f =7  # log10 rate constant (ref=1e7) 
    log_k3_r =3  # log10 rate constant (ref=1e3)  
    log_k4_f ~ Uniform(-1, 5) # log10 rate constant (ref=1e3) 
    log_k4_r ~ Uniform(6, 12)  # log10 rate constant (ref=1e10)
    log_k5_f =2  # log10 rate constant (ref=1e2)
    log_k5_r =2   # log10 rate constant (ref=1e2)
    log_k6_f =3  # log10  rate constant (ref=1e3)
    
    # set parameter values and initial concentrations
    log_k6_r = (log_k1_f+log_k2_f+log_k3_f+log_k4_f+log_k5_f+log_k6_f)-(log_k1_r+log_k2_r+log_k3_r+log_k4_r+log_k5_r) 
    pmap2 = [H_out => H_out_ref, S_out => S_out_ref,
            k1_f =>10.0^log_k1_f, k1_r =>10.0^log_k1_r,
            k2_f =>10.0^log_k2_f, k2_r =>10.0^log_k2_r,
            k3_f =>10.0^log_k3_f, k3_r =>10.0^log_k3_r,
            k4_f =>10.0^log_k4_f, k4_r =>10.0^log_k4_r,
            k5_f =>10.0^log_k5_f, k5_r =>10.0^log_k5_f,
            k6_f =>10.0^log_k6_f, k6_r =>10.0^log_k6_r,
    ]
    
    # solve ODE system
    pnew2 = varmap_to_vars(pmap2, parameters(sys))
    oprob2 = remake(prob, p=pnew2)
    sol2 = solve(oprob2, TRBDF2(), abstol = 1e-18, reltol = 1e-12, saveat=0.04, callback=cb, tstops=event_times)
    
    # calc flux (y_pred)
    k2 = [pmap2[9][2], pmap2[10][2]]
    c2 = [sol2[IF_Hb_Sb], sol2[H_in], sol2[IF_Sb]]
    vol = 0.0001
    y_pred2 = calc_flux(k2,c2,vol)
    
    for i = 1:length(data)
        data[i] ~ Normal(y_pred2[i], stdev_sample2)
    end
end
model = rxn_sampler(y_obs, oprob, odesys)

#p2 = [10.1, 3.1]
    
iterations = 1_000
#chain = sample(model, NUTS(0.45), iterations, init_params=p2)
chain = sample(model, NUTS(0.45), iterations)
#chain = mapreduce(c -> sample(model, NUTS(.65),iterations, init_params=p2), chainscat, 1:3)
#chain = mapreduce(c -> sample(model, MH(), iterations), chainscat, 1:3)

#c_size = auto_tune_chunk_size!(model::Function, rep_num=10)
#println(c_size)
##### end of sampling


┌ Info: [Turing]: AD chunk size is set as 400
└ @ Turing.Essential /Users/georgeau/.julia/packages/Turing/rl6ku/src/essential/ad.jl:33
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/georgeau/.julia/packages/AdvancedHMC/kB7Xa/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/georgeau/.julia/packages/AdvancedHMC/kB7Xa/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/georgeau/.julia/packages/AdvancedHMC/kB7Xa/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/georgeau/.julia/packages/AdvancedH

378
10792.217228574395
2D grid...10792.217228577721
sampling...


Chains MCMC chain (1000×14×1 Array{Float64, 3}):

Iterations        = 501:1:1500
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 66.06 seconds
Compute duration  = 66.06 seconds
parameters        = log_k4_f, log_k4_r
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse       ess      rhat   es ⋯
      Symbol   Float64   Float64    Float64   Float64   Float64   Float64      ⋯

    log_k4_f    2.4198    0.0025     0.0001    0.0004    2.2078    2.7424      ⋯
    log_k4_r    8.2394    0.0498     0.0016    0.0088    2.1977    2.8056      ⋯
                                                                1 column omitted

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float

In [ ]:
### plot log likelihood
heatmap(x1_range,x2_range,z, clim=(max_log_like-20, max_log_like))
plot!(size=(500,500))

In [ ]:
savefig("logk1f_logk1r_100x_heatmap_all_data_1x_stdev_full.png")

In [ ]:
### plot 2d marginals
full = true
max_lp = -Inf
lp = chain[:lp]
for i in 1:length(lp)
    if lp[i]>max_lp
        max_lp = lp[i]
    end
end    
println(max_lp)
if full == true
    corner(chain, ylim=[-1,5], xlim=[6,12])
else
    corner(chain)
end

In [ ]:
if full == true
    savefig("logk4f_logk4r_1000_NUTS_045_full.png")
else
    savefig("logk4f_logk4r_1000_NUTS_045_auto.png")
end





In [ ]:
plot(chain)
